In [ ]:
import pandas as pd
ALARM_FILE = "alarm.viewer.F4.09.xlsx"

In [ ]:
import pandas as pd

# 1) Load the file
df = pd.read_excel(ALARM_FILE, engine='openpyxl')

# 2) Keep only rows with '313PT0' in TagName
alarm = df[df['TagName'].str.contains('313PT0', na=False)].copy()

# 3) Keep only HiHi/LoLo limit alarms
alarm = alarm[alarm['MessageText']
              .str.contains(r'HiHi level|LoLo level', regex=True, na=False)].copy()

# 4) Keep only the two columns we care about
alarm = alarm[['DateTime', 'TagName']].copy()

from dateutil import parser
import pandas as pd

# ... after steps 1–4, before you split TagName …

def robust_parse(x):
    if pd.isna(x):
        return pd.NaT
    # If Excel gave you a numeric serial date:
    if isinstance(x, (int, float)):
        # Excel’s “day 1” is 1899-12-31 (but pandas uses 1899-12-30 as origin)
        return pd.to_datetime('1899-12-30') + pd.to_timedelta(x, unit='D')
    # Otherwise parse any string, day-first, with or without ms
    return parser.parse(str(x), dayfirst=True)

alarm['DateTime'] = alarm['DateTime'].apply(robust_parse)


# 6) Split TagName into 'asset' and 'alarm'
parts = alarm['TagName'].str.split('.', expand=True)
alarm['asset'] = parts[0] + '.' + parts[1]
alarm['alarm'] = parts[2]

# 7) Reorder and reset index
alarm = alarm[['DateTime', 'asset', 'alarm']].reset_index(drop=True)

print(alarm)


In [ ]:
# Extract the digits after '313PT0' and convert to int
alarm['asset'] = alarm['asset'].str.extract(r'313PT0(\d+)', expand=False).astype(int)
print(alarm)


In [ ]:
import pandas as pd

# If not already datetime dtype:
alarm['DateTime'] = pd.to_datetime(alarm['DateTime'])

# Round up to nearest 5 seconds
alarm['DateTime'] = alarm['DateTime'].dt.ceil('5S')
print(alarm)

In [ ]:
import pyodbc
import time
from datetime import datetime
import configparser
import os
import pandas as pd

# Load configuration from input_config.ini
config = configparser.ConfigParser()
config.read('input_data.ini')

# Define tags and query settings to be fetched from config file
tags = config.get('QUERY', 'tags').split(', ')
start_datetime = config.get('QUERY', 'start_datetime')
end_datetime = config.get('QUERY', 'end_datetime')
# (csv_directory and csv_filename are no longer strictly needed)
csv_directory = config.get('OUTPUT', 'csv_directory')
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = os.path.join(csv_directory, f"output_data_{current_time}.csv")

# Define query settings
ww_retrieval_mode = config.get('QUERY_SETTINGS', 'ww_retrieval_mode', fallback='Cyclic')
ww_resolution = config.getint('QUERY_SETTINGS', 'ww_resolution', fallback=10000)
ww_quality_rule = config.get('QUERY_SETTINGS', 'ww_quality_rule', fallback='Extended')
ww_version = config.get('QUERY_SETTINGS', 'ww_version', fallback='Latest')
production_mode_threshold = config.getint('QUERY_SETTINGS', 'production_mode_threshold', fallback=5)

def fetch_data(tags, start_datetime, end_datetime):
    try:
        conn_str = (
            r'Driver={SQL Server};'
            r'Server=IJMHISDBS03.EDIS.TATASTEEL.COM;'
            r'Database=Runtime;'
            r'Trusted_Connection=yes;'
        )
        connection = pyodbc.connect(conn_str)
        cursor = connection.cursor()

        print('Fetching data...')

        tags_str = ", ".join(f"[{tag}]" for tag in tags)
        query = (
            f"SET QUOTED_IDENTIFIER OFF "
            f"SELECT * FROM OPENQUERY(INSQL, \"SELECT "
            f"DateTime = convert(nvarchar, DateTime, 21), "
            f"{tags_str} "
            f"FROM WideHistory "
            f"WHERE wwRetrievalMode = '{ww_retrieval_mode}' "
            f" AND wwResolution = {ww_resolution} "
            f" AND wwQualityRule = '{ww_quality_rule}' "
            f"AND wwVersion = '{ww_version}' "
            f"AND DateTime >= '{start_datetime}' "
            f"AND DateTime <= '{end_datetime}' "
            f"AND [PlantInformation.ProductionMode] >= {production_mode_threshold} \" )"
        )
        
        cursor.execute(query)
        while True:
            data_chunk = cursor.fetchmany(1000)  # Fetch 1,000 rows at a time
            if not data_chunk:
                break
            yield data_chunk
            print(f"Fetched {len(data_chunk)} records...")
        cursor.close()

    except pyodbc.Error as e:
        print(f"Failed to fetch data:\n{str(e)}")
        return

def main():
    all_rows = []
    # Fetch data in chunks and accumulate into a list
    for data_chunk in fetch_data(tags, start_datetime, end_datetime):
        if data_chunk:
            all_rows.extend(data_chunk)
    # Build DataFrame, converting each pyodbc.Row into columns correctly
    trend_ = pd.DataFrame.from_records(all_rows, columns=["DateTime"] + tags)
    return trend_

if __name__ == "__main__":
    trend_ = main()
    print(trend_)


In [ ]:
# 1. Read your original CSV
df = pd.read_csv(TREND_FILE)

# 2. Specify which column(s) to keep fixed (ID variables).
#    Here we assume the first column is your timestamp or key.
id_vars = [df.columns[0]]

# 3. Melt (unpivot) everything else into two columns: 'Variable' and 'Value'
trend = trend_.melt(
    id_vars=id_vars,
    var_name='Variable',
    value_name='Value'
)

# 4. (Optional) Reorder or rename columns
#    e.g., df_unpivoted = df_unpivoted[['Timestamp','Variable','Value']]

# 5. Save the result
trend.to_csv('historian.db.F5.03_unpivot.csv', index=False)

# 6. Quick check
print(trend.head())


In [ ]:
# Rename the columns as requested
trend.rename(
    columns={
        'Variable': 'asset',
        'Value': 'valve pos'
    },
    inplace=True
)

# Verify the rename
print(trend.head())

In [ ]:
# Assuming `trend` exists with an 'asset' column like '313XV021.Status'
trend['asset'] = trend['asset'] \
    .str.extract(r'(\d+)\.Status') \
    .astype(int) - 20

# Verify the transformation
print(trend)


In [ ]:
# 1. Ensure DateTime columns are proper datetime dtype
trend['DateTime'] = pd.to_datetime(trend['DateTime'])
alarm['DateTime'] = pd.to_datetime(alarm['DateTime'])

# 2. Merge on both 'asset' and 'DateTime' (left join to keep all trend rows)
merged_df = pd.merge(
    trend,
    alarm,
    on=['asset', 'DateTime'],
    how='left'
)

# 3. (Optional) Fill non-matching alarm entries if desired
merged_df['alarm'] = merged_df['alarm']

# 4. Quick check
print(merged_df.head())


In [ ]:
merged_df.count()

In [ ]:
# assuming you’ve already done: import pandas as pd

# Map 'HiHi' → 1 and 'LoLo' → -1, overwriting the existing column
merged_df['alarm'] = merged_df['alarm'].map({'HiHi': 1, 'LoLo': -1}).fillna(0)

print(merged_df)

In [ ]:
import pandas as pd
import numpy as np

# ensure DateTime is a datetime dtype and df is sorted
merged_df['DateTime'] = pd.to_datetime(merged_df['DateTime'])
merged_df = merged_df.sort_values(['asset', 'DateTime'])

# prepare the output column
merged_df['failure_period_HiHi'] = 0

# work asset by asset
for asset in merged_df['asset'].unique():
    idx = merged_df['asset'] == asset
    group = merged_df.loc[idx]
    
    ts    = group['DateTime'].values
    alarm = group['alarm'].values
    valve = group['valve pos'].values

    # precompute “time since start of current valve==2 run”
    is_v2 = (valve == 2)
    # every time valve≠2 we bump the run-id
    run2 = (~is_v2).cumsum()
    # first timestamp of each run
    first2 = pd.Series(ts).groupby(run2).transform('first').values
    since2 = (pd.Series(ts) - first2).dt.total_seconds().values

    in_failure = False
    # iterate in chronological order
    for i, loc in enumerate(group.index):
        if not in_failure:
            # 1) look for alarm==1, then valve==1 within next 5 s
            if alarm[i] == 1:
                t0 = ts[i]
                # boolean mask of rows within (t0, t0+5s] with valve==1
                # we can limit to the next few seconds since data is 1 Hz
                mask = (ts > t0) & (ts <= t0 + np.timedelta64(5, 's')) & (valve == 1)
                if mask.any():
                    in_failure = True
                    merged_df.at[loc, 'failure_period_HiHi'] = 1
        else:
            # 2) we’re in a failure—check for shutdown condition
            #    “valve==2 continuously for ≥10 s” resets failure
            if is_v2[i] and (since2[i] >= 10):
                in_failure = False
                # leave this row at 0
            else:
                merged_df.at[loc, 'failure_period_HiHi'] = 1

# merged_df is still sorted by ['asset','DateTime'], with the new column added


In [ ]:
merged_df

In [ ]:
import pandas as pd
from dateutil import parser

# helper to catch serials and varied strings
def robust_parse(x):
    if pd.isna(x):
        return pd.NaT
    # Excel serial date → Timestamp
    if isinstance(x, (int, float)):
        # pandas uses 1899-12-30 as day 0
        return pd.to_datetime('1899-12-30') + pd.to_timedelta(x, unit='D')
    # otherwise parse with day-first
    return parser.parse(str(x), dayfirst=True)

# 1) Load the file
df = pd.read_excel(ALARM_FILE, engine='openpyxl')

# 2) Keep only rows with '140M0' in TagName
alarm = df[df['TagName'].str.contains('140M0', na=False)].copy()

# 3) Keep only Drive alarms
alarm = alarm[
    alarm['MessageText']
         .str.contains(r'Drive alarm', regex=True, na=False)
].copy()

# 4) Keep only the two columns we care about
alarm = alarm[['DateTime', 'TagName']].copy()

# 4b) If there were no alarms, give ourselves an empty frame with the right columns
if alarm.empty:
    alarm = pd.DataFrame({
        'DateTime':        pd.Series(dtype='datetime64[ns]'),
        'asset':           pd.Series(dtype='object'),
        'drive warning':   pd.Series(dtype='int8'),
    })
else:
    # 5) Robustly parse any format (serial, with or without ms) then floor to seconds
    alarm['DateTime'] = (
        alarm['DateTime']
             .apply(robust_parse)
             .dt.floor('S')
    )

    # 6) Split out the asset, set the flag
    parts = alarm['TagName'].str.split('.', expand=True)
    alarm['asset'] = parts[0]
    alarm['drive warning'] = 1

    # 7) Keep only the bits we want
    alarm = alarm[['DateTime', 'asset', 'drive warning']]

# 8) Reset index in any case
alarm = alarm.reset_index(drop=True)

print(alarm)


In [ ]:
# Extract the digits after '313PT0' and convert to int
alarm['asset'] = alarm['asset'].str.extract(r'140M0(\d+)', expand=False).astype(int)
print(alarm)


In [ ]:
# 1. Ensure DateTime columns are proper datetime dtype
merged_df['DateTime'] = pd.to_datetime(merged_df['DateTime'])
alarm['DateTime'] = pd.to_datetime(alarm['DateTime'])

# 2. Merge on both 'asset' and 'DateTime' (left join to keep all trend rows)
merged_merged_df = pd.merge(
    merged_df,
    alarm,
    on=['asset', 'DateTime'],
    how='left'
)

merged_merged_df['drive warning'] = merged_merged_df['drive warning'].fillna(0)

# 4. Quick check
print(merged_merged_df.head())


In [ ]:
merged_merged_df.to_csv('merged_df.csv', index=False)

In [ ]:
# count the number of 1’s in the ‘failure_period_HiHi’ column
count_ones = (merged_merged_df['alarm'] == 1).sum()
print(f"Number of 1’s in failure_period_HiHi: {count_ones}")


In [ ]:
import pandas as pd

# Work on a copy of your existing DataFrame
df = merged_merged_df.copy()

# 1) Parse your time column (adjust name if needed)
time_col = "DateTime"  # or "time", "date", etc.
df[time_col] = pd.to_datetime(df[time_col])

# 2) Sort & group by asset
df = df.sort_values([time_col, "asset"])
grp = df.groupby("asset")

# 3) Helper to count transitions A→B
def count_transitions(s, A, B):
    return ((s.shift(1) == A) & (s == B)).sum()

# 4) Compute counts per asset
high_pressure = grp["failure_period_HiHi"]\
    .apply(lambda s: count_transitions(s, 0, 1))\
    .rename("High pressure failures")

low_pressure = grp["alarm"]\
    .apply(lambda s: count_transitions(s, 0, -1))\
    .rename("Low pressure failures")

rotary_feeder = grp["drive warning"]\
    .apply(lambda s: count_transitions(s, 0, 1))\
    .rename("Rotary feeder failures")

# 5) Compute minutes from first 0→1 to next return-to-0 in failure_period_HiHi
def first_failure_duration(df_asset):
    ser = df_asset.sort_values(time_col)["failure_period_HiHi"]
    times = df_asset.sort_values(time_col)[time_col]
    # find first 0→1
    mask_start = (ser.shift(1) == 0) & (ser == 1)
    if not mask_start.any():
        return pd.NA
    start_idx = mask_start.idxmax()
    start_time = times.loc[start_idx]
    # find next 0 after that
    post = ser.loc[start_idx+1:]
    zeros = post[post == 0]
    if zeros.empty:
        return pd.NA
    end_idx = zeros.index[0]
    end_time = times.loc[end_idx]
    return (end_time - start_time).total_seconds() / 60.0

durations = grp.apply(first_failure_duration)\
               .rename("failure time HiHi injector (min)")

# 6) Combine into summary DataFrame
summary = pd.concat([
    high_pressure,
    low_pressure,
    rotary_feeder,
    durations
], axis=1).reset_index()

print(summary)


In [ ]:
from IPython.display import display

# summary is the DataFrame you computed
display(summary)



In [ ]:
# export to Excel
summary.to_excel("failures_summary.xlsx", index=False)